In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Nalaganje podatkovne zbirke slik ušes

# New Section

In [6]:
import os
import csv
import cv2
import keras

TRAINING_CSV_FILE = '/content/drive/MyDrive/biomet/training_data.csv'
TRAINING_IMAGE_DIR = '/content/drive/MyDrive/biomet/awe/data/ears/train/'
WIDTH, HEIGHT = 480, 360

f = open('/content/drive/MyDrive/biomet/awe/data/ears/annotations/detection/train.txt', 'r')

train_targets = []
train_images = []

line = f.readline()
while line:
  data = line.split(' ')
  img_path = data[0].split('/')[1]

  train_image_fullpath = TRAINING_IMAGE_DIR + img_path
  train_images.append(cv2.imread(train_image_fullpath, cv2.IMREAD_GRAYSCALE)/255)

  xmin = round(int(data[1]) / WIDTH, 2)
  ymin = round(int(data[2]) / HEIGHT, 2)
  xmax = round((int(data[3]) + int(data[1])) / WIDTH, 2)
  ymax = round((int(data[4]) + int(data[2])) / HEIGHT, 2)

  train_targets.append([ xmin, ymin, xmax, ymax])

  line = f.readline()

f.close()

In [7]:
VAL_IMAGE_DIR = '/content/drive/MyDrive/biomet/awe/data/ears/test/'
WIDTH, HEIGHT = 480, 360

f = open('/content/drive/MyDrive/biomet/awe/data/ears/annotations/detection/test.txt', 'r')

val_targets = []
val_images = []

line = f.readline()
while line:
  data = line.split(' ')
  img_path = data[0].split('/')[1]

  val_image_fullpath = VAL_IMAGE_DIR + img_path
  val_images.append(cv2.imread(val_image_fullpath, cv2.IMREAD_GRAYSCALE)/255)

  xmin = round(int(data[1]) / WIDTH, 2)
  ymin = round(int(data[2]) / HEIGHT, 2)
  xmax = round((int(data[3]) + int(data[1])) / WIDTH, 2)
  ymax = round((int(data[4]) + int(data[2])) / HEIGHT, 2)

  val_targets.append([xmin, ymin, xmax, ymax])

  line = f.readline()

f.close()

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib
import pandas as pd
from PIL import Image 
from PIL.ImageDraw import Draw

#create the common input layer
input_shape = (HEIGHT, WIDTH, 1)
input_layer = tf.keras.layers.Input(input_shape)
labels = layers.Input(name="label", shape=(None,), dtype="float32")

#create the base layers
base_layers = layers.Conv2D(16, (3, 3), padding='same', activation='relu', name='bl_1')(input_layer)
base_layers = layers.MaxPooling2D((2, 2), name='bl_2')(base_layers)
base_layers = layers.Conv2D(32, (3, 3), padding='same', activation='relu', name='bl_3')(base_layers)
base_layers = layers.MaxPooling2D((2, 2), name='bl_4')(base_layers)
base_layers = layers.Conv2D(64, (3, 3), padding='same', activation='relu', name='bl_5')(base_layers)
base_layers = layers.MaxPooling2D((2, 2), name='bl_6')(base_layers)
base_layers = layers.Flatten(name='bl_7')(base_layers)

#create the localiser branch
locator_branch = layers.Dense(128, activation='relu', name='bb_1')(base_layers)
locator_branch = layers.Dense(64, activation='relu', name='bb_2')(locator_branch)
locator_branch = layers.Dense(32, activation='relu', name='bb_3')(locator_branch)
locator_branch = layers.Dense(4, activation='sigmoid', name='bb_head')(locator_branch)

In [9]:
model = tf.keras.Model(input_layer, outputs=locator_branch)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 360, 480, 1)]     0         
                                                                 
 bl_1 (Conv2D)               (None, 360, 480, 16)      160       
                                                                 
 bl_2 (MaxPooling2D)         (None, 180, 240, 16)      0         
                                                                 
 bl_3 (Conv2D)               (None, 180, 240, 32)      4640      
                                                                 
 bl_4 (MaxPooling2D)         (None, 90, 120, 32)       0         
                                                                 
 bl_5 (Conv2D)               (None, 90, 120, 64)       18496     
                                                                 
 bl_6 (MaxPooling2D)         (None, 45, 60, 64)        0     

In [18]:
import numpy as np

train_images = np.array(train_images)
val_images = np.array(val_images)
train_targets = np.array(train_targets)
val_targets = np.array(val_targets)

In [11]:
INIT_LR = 1e-4
NUM_EPOCHS = 55
BATCH_SIZE = 15

opt = tf.keras.optimizers.Adam()
losses = {"bb_head":tf.keras.losses.MSE}

model.compile(loss=tf.keras.losses.MSE, optimizer='Adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_targets, validation_data=(val_images, val_targets), batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, shuffle=True, verbose=1)

Epoch 1/55
57/57 [==============================] - 133s 2s/step - loss: 0.0363 - accuracy: 0.5576 - val_loss: 0.0337 - val_accuracy: 0.5694
Epoch 2/55
57/57 [==============================] - 132s 2s/step - loss: 0.0298 - accuracy: 0.6212 - val_loss: 0.0298 - val_accuracy: 0.6319
Epoch 3/55
57/57 [==============================] - 132s 2s/step - loss: 0.0236 - accuracy: 0.7153 - val_loss: 0.0235 - val_accuracy: 0.7292
Epoch 4/55
57/57 [==============================] - 133s 2s/step - loss: 0.0187 - accuracy: 0.7776 - val_loss: 0.0246 - val_accuracy: 0.6319
Epoch 5/55
57/57 [==============================] - 132s 2s/step - loss: 0.0160 - accuracy: 0.8059 - val_loss: 0.0252 - val_accuracy: 0.6910
Epoch 6/55
57/57 [==============================] - 132s 2s/step - loss: 0.0124 - accuracy: 0.8306 - val_loss: 0.0237 - val_accuracy: 0.7083
Epoch 7/55
57/57 [==============================] - 132s 2s/step - loss: 0.0108 - accuracy: 0.8471 - val_loss: 0.0222 - val_accuracy: 0.7326
Epoch 8/55
57

In [ ]:
img = cv2.resize(cv2.imread('/content/drive/MyDrive/biomet/awe/data/ears/adam.jpg'), (width, height)) / 255
pred = model.predict(np.array([img]))[0] * 1200 //1

pred

In [ ]:
history.history['val_accuracy']